In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd
import numpy as np
import random
import json
import dask.dataframe as dd
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

In [3]:
dtype = {'user_ratings_total': 'Int64'}
df_tour = pd.read_csv('/gdrive/Shareddrives/從資料學習2/Dataset & code/cleaning data/restaurant_data_output.csv', dtype=dtype)

In [4]:
df_tour.head()

,place_id,name,business_status,wheelchair_accessible_entrance,type,rating,editorial_summary,user_ratings_total,price_level,formatted_phone_number,...,dine_in,delivery,"location(lat,lng)",reviews_text,serves_breakfast,serves_brunch,serves_lunch,serves_dinner,serves_vegetarian_food,keywords
0,ChIJt9661tCrQjQRXMVpa4A5Bmg,L' AGE 熟成餐廳,OPERATIONAL,NaN,"['restaurant', 'food', 'point_of_interest', 'e...",4.6,NaN,505,2.0,02 2711 3881,...,True,False,"{'lat': 25.0396075, 'lng': 121.5431817}",['第一次拜訪，每道菜大致上口味都很不錯，但有幾道餐點的份量偏少，覺得和價格不成正比，花的費...,False,False,True,True,NaN,"['餐點', '用餐', '沒有', '兩道', '推薦', '主廚', '處理', '親切..."
1,ChIJtxAbUsSrQjQRnqXBc75y9kE,莫夏67印度餐廳大安店 Moksha 67 indian restaurant-daan b...,OPERATIONAL,NaN,"['restaurant', 'point_of_interest', 'food', 'e...",4.7,NaN,631,NaN,02 2706 9878,...,True,True,"{'lat': 25.0369707, 'lng': 121.5469293}",['四人用餐，點的每一道都讓大家讚不絕口：\n\n✨酥炸脆球餅、鑲餡蘑菇、串烤起司去骨雞、茴...,NaN,NaN,True,True,True,"['烤餅', '咖哩', '味道', '香料', '奶茶', '球餅', '喜歡', '有點..."
2,ChIJJXnO0barQjQR0ZJz1JGJ8v4,馬友友印度餐廳酒吧+水煙館,OPERATIONAL,True,"['bar', 'restaurant', 'point_of_interest', 'fo...",4.7,"{'language': 'zh-TW', 'overview': '供應印度與中式餐點的新...",2118,NaN,02 2500 6186,...,True,True,"{'lat': 25.0458202, 'lng': 121.5332625}",['特色金邊餐桌椅，進去有小酒館風。有水煙，晚上會有音樂，歌唱表演。我們中午用餐。點了玉米濃...,NaN,False,True,True,True,"['香料', '雞肉', '咖哩', '烤餅', '風格', '濃湯', '還有', '味道..."
3,ChIJ0RrGQ8irQjQR3tnVYowd0w8,籌學費餐廳,OPERATIONAL,False,"['restaurant', 'food', 'point_of_interest', 'e...",4.5,NaN,649,2.0,02 2345 1398,...,True,True,"{'lat': 25.0369568, 'lng': 121.557991}",['營業到比較晚，晚歸的晚餐選擇。有附茶。\n\n▲滷味：非常好吃，建議直接點大份。\n▲墨...,False,False,True,True,NaN,"['牛肉', '服務費', '味道', '不錯', '大利', '非常', '建議', '中..."
4,ChIJA35dJHyrQjQRJ-sD3kSwWUY,Mellow Bar & Restaurant-東區精釀啤酒|聚餐餐廳|必吃餐酒館|網美餐酒...,OPERATIONAL,NaN,"['restaurant', 'point_of_interest', 'food', 'e...",4.9,NaN,515,NaN,02 2771 9907,...,True,False,"{'lat': 25.0397848, 'lng': 121.5522774}",['餐點用心，感受到主廚實力跟用心，炸物沒有油垢味、長島冰啤讚、千層義大利麵超棒、起司牛肉堡...,NaN,NaN,NaN,True,NaN,"['推薦度', '奶油', '餐點', '沒有', '不錯', '明顯', '紫蘇', '店..."


In [5]:
import networkx as nx
import pandas as pd

# 載入資料
df = pd.read_csv('/gdrive/Shareddrives/從資料學習2/Dataset & code/cleaning data/restaurant_data_output.csv', dtype=dtype)

# 建立雙向無圈圖
G = nx.Graph()

# 加入節點和邊
for _, row in df.iterrows():
    restaurant_node = row['name']
    G.add_node(restaurant_node, bipartite=1)  # 餐廳節點
    for keyword in row['keywords']:
        keyword_node = keyword
        G.add_node(keyword_node, bipartite=0)  # 關鍵字節點
        G.add_edge(restaurant_node, keyword_node)  # 建立邊

# 推薦餐廳函數
def get_restaurant_recommendations(chosen_restaurant, G):
    recommendations = []

    # 找到所選餐廳的節點
    chosen_node = [node for node in G.nodes if chosen_restaurant in node][0]

    # 找到與所選餐廳相連的關鍵字節點
    keywords = [neighbor for neighbor in G.neighbors(chosen_node) if G.nodes[neighbor]['bipartite'] == 0]

    # 找到與這些關鍵字相連的其他餐廳節點
    for keyword in keywords:
        restaurants = [neighbor for neighbor in G.neighbors(keyword) if G.nodes[neighbor]['bipartite'] == 1]
        recommendations.extend(restaurants)

    # 移除所選餐廳
    recommendations = [restaurant for restaurant in recommendations if restaurant != chosen_node]

    return list(set(recommendations))

# 使用範例
chosen_restaurant = "馬友友印度餐廳酒吧+水煙館"
recommendations = get_restaurant_recommendations(chosen_restaurant, G)
print(f"對於 '{chosen_restaurant}' 的推薦餐廳: {recommendations}")

對於 '馬友友印度餐廳酒吧+水煙館' 的推薦餐廳: ['錢都日式涮涮鍋-新莊中華店 CHIEN TU HOT POT', '當歸羊鴨肉', '家圓', '楊家菜館', '燒仙草', 'Q Burger 新莊民安店', '約好。守食', 'Flavor of India Vegetarian Restaurant 品·印度蔬食餐廳', '義美見學餐廳', 'OH LA LA 法式薄餅 · 家常菜 French Crêperie', '阿志牛肉麵', '愛民飯店', '合歡茶宴風味餐廳', '河內越南料理', '古窗海景會館-瑞芳景觀餐廳 九份美食 九份餐廳 九份必吃 九份restaurant 九份民宿', '拉茲印度音樂餐廳', '竹逸樓餐館', '橋園食坊', '仁愛潤餅捲', '好再來餐館', 'Restaurant', 'すき家國父紀念館', '大同早餐店', '一家。麵食—全素 素.緣.蔬食', 'PastaHome 我家義大利麵', '半島牛肉麵', '聽雨軒 壽司鐵板（潼日式料理）', '蔬軾．樂華店．創意蔬食料理．So Veg Restaurant', '羊之屋餐廳', 'South Beauty Restaurant', '山背家菜館', '百分百早餐店', '舊道口牛肉麵', 'PAPA RESTAURANT /師大美食/師大義大利麵/師大簡餐/師大披薩/師大手工烘培/師大歐式麵包/師大下午茶/師大聚餐推薦/師大商業午餐/師大包場', '呂壽司-南港美食-餐廳-生魚片-Restaurants', 'Oval meat 美式餐廳', 'LUCKY CLOVER CAFE & ASIAN CUISINE', '潼 鐵板料理至善店', '阿英草仔粿', 'Sunny Q 陽光公主 義國餐廳', '烤就對了 雞本人桶仔雞', '養心茶樓蔬食飲茶(名人堂花園大飯店) Yang Shin Vegetarian Restaurant', 'RAW', "Marco Pizza Jacob's store", 'GIRAR bar & restaurant', '蓉渝川菜', '拾山享中餐廳', 'Boom Burger', "Na'vi Garden Restaurant", '野柳望海亭海鮮餐廳', 'すき家 SUKIYA 三峽店', '舜德農莊休閒餐廳', '

In [7]:
import networkx as nx
import pandas as pd
import random

# 載入資料
df = pd.read_csv('/gdrive/Shareddrives/從資料學習2/Dataset & code/cleaning data/restaurant_data_output.csv', dtype=dtype)

# 建立雙向無圈圖
G = nx.Graph()

# 加入節點和邊
for _, row in df.iterrows():
    restaurant_node = row['name']
    G.add_node(restaurant_node, bipartite=1)  # 餐廳節點
    for keyword in row['keywords']:
        keyword_node = keyword
        G.add_node(keyword_node, bipartite=0)  # 關鍵字節點
        G.add_edge(restaurant_node, keyword_node)  # 建立邊

# 推薦餐廳函數
def get_restaurant_recommendations(chosen_restaurant, G, num_recommendations=1000):
    recommendations = []

    # 找到所選餐廳的節點
    chosen_node = [node for node in G.nodes if chosen_restaurant in node][0]

    # 找到與所選餐廳相連的關鍵字節點
    keywords = [neighbor for neighbor in G.neighbors(chosen_node) if G.nodes[neighbor]['bipartite'] == 0]

    # 找到與這些關鍵字相連的其他餐廳節點
    for keyword in keywords:
        restaurants = [neighbor for neighbor in G.neighbors(keyword) if G.nodes[neighbor]['bipartite'] == 1]
        recommendations.extend(restaurants)

    # 移除所選餐廳
    recommendations = [restaurant for restaurant in recommendations if restaurant != chosen_node]

    # 隨機選擇num_recommendations個推薦餐廳
    recommendations = random.sample(recommendations, min(num_recommendations, len(recommendations)))

    return recommendations

# 使用範例
chosen_restaurant = "馬友友印度餐廳酒吧+水煙館"
recommendations = get_restaurant_recommendations(chosen_restaurant, G)
print(f"對於 '{chosen_restaurant}' 的推薦餐廳:")
for restaurant in recommendations:
    print(restaurant)

對於 '馬友友印度餐廳酒吧+水煙館' 的推薦餐廳:
知味鄉玉米
印度先生
順意素食小吃
すき家板橋新埔
雲朵Cloud 義大利麵 早午餐
古道客棧
真好味烤鴨莊
阿婆麵店
昶帝嶺餐廳
鑫華茶餐廳
卯澳灣食堂
阿妹の店
貝兒義式餐廳
來來活海鮮
Churchill's Sausages
荒郊野外土雞城
貓空 四爺
東悅ㄧ碗小肥羊
紫艷中餐廳
金蓬萊遵古台菜餐廳
雅竹居酒屋
馬友友印度廚房 - 大直 Mayur Indian Kitchen restaurant Dazhi (MiK-hi5)
黃麵
Dennis 印度餐廳Indian Restaurant
4分之一restaurant. ( Taiwan Way )
無名鴨肉
Cheers 乾杯吧 Linkou
三野達人
大清早餐店
Bella Roma ltalian WINE Restaurant
好姐妹
好滋味土雞莊
山羊樓餐廳
YAYOI彌生軒 七張店
阿蝦古早味麵店
Restaurant Go.老吳的西洋料理
當歸羊鴨肉
祥豐市場130黑白切
千葉日式居酒屋
Juicy Bun Burger 就是棒 美式餐廳 政大店
MAGiC TOUCH点爭鮮 CITYLINK-南港店｜日式料理壽司生魚片
海邊灶咖 Ocean Kitchen & Cafe Shop
築間幸福鍋物 基隆新豐店
尚豪涼麵
麥當勞-嘉義北港餐廳
Sea seafood fresh seafood
June Burger 啾漢堡店.早午餐.寵物友善+可落地.信義區美式早餐.經典美式漢堡.永春站早餐
焱炸雞麻豆店
Momo將串燒
滇緬小鎮
麗寶樂園 恐龍主題餐廳
麥當勞-新店安康餐廳
Anchovy restaurant(鯷魚餐廳）
Little Beanie Chinese Quick Service Restaurant
安德魯印度料理
來一碗（甜不辣 肉羹）
華之友自然景觀農園
甲鼎川菜館
三姊妹早餐店
素食茄汁牛肉麵
敏忠餐廳
觀世音素食
台灣山豬城
舊道口牛肉麵
欣葉台菜 信義新天地A9店
法蘭西早午餐
饗食天堂 台北京站店
食為天酒樓
社皮美早餐店
奧特拉麵Ramen Ultra 南港環球店 (錦拉麵)B1
Rendang House
東木河茶莊
魚舞和食居酒屋
NuBroiler
鐵道燒烤屋@宵夜，聚餐，小酌
海龍珠海鮮餐廳
土托魚羹

In [9]:
import networkx as nx
import pandas as pd
import random
import ast

# 讀取資料
# 確保 `keywords` 欄位是列表格式
df['keywords'] = df['keywords'].apply(ast.literal_eval)

# 建立雙向無圈圖
G = nx.Graph()

# 加入節點和邊
for _, row in df.iterrows():
    restaurant_node = row['name']
    G.add_node(restaurant_node, bipartite=1)  # 餐廳節點
    for keyword in row['keywords']:
        keyword_node = keyword
        G.add_node(keyword_node, bipartite=0)  # 關鍵字節點
        G.add_edge(restaurant_node, keyword_node)  # 建立邊

# 推薦餐廳函數
def get_restaurant_recommendations(chosen_restaurant, G, num_recommendations=1000):
    recommendations = []

    # 確認所選餐廳存在於圖中
    if chosen_restaurant not in G:
        print(f"餐廳 '{chosen_restaurant}' 不存在於數據中。")
        return recommendations

    # 找到與所選餐廳相連的關鍵字節點
    keywords = [neighbor for neighbor in G.neighbors(chosen_restaurant) if G.nodes[neighbor]['bipartite'] == 0]

    # 找到與這些關鍵字相連的其他餐廳節點
    for keyword in keywords:
        restaurants = [neighbor for neighbor in G.neighbors(keyword) if G.nodes[neighbor]['bipartite'] == 1]
        recommendations.extend(restaurants)

    # 移除所選餐廳
    recommendations = [restaurant for restaurant in recommendations if restaurant != chosen_restaurant]

    # 隨機選擇num_recommendations個推薦餐廳
    if recommendations:
        recommendations = random.sample(recommendations, min(num_recommendations, len(recommendations)))
    else:
        print("無法根據所選餐廳找到推薦餐廳。")

    return recommendations

# 使用範例
chosen_restaurant = "馬友友印度餐廳酒吧+水煙館"
recommendations = get_restaurant_recommendations(chosen_restaurant, G)
print(f"對於 '{chosen_restaurant}' 的推薦餐廳:")
for restaurant in recommendations:
    print(restaurant)


對於 '馬友友印度餐廳酒吧+水煙館' 的推薦餐廳:
新卡莎素食西餐廳
台之最餐廳
尚品早餐店
酌悅居酒屋
拉合爾 印度/巴基斯坦異國美食
印度之夏印度餐廳 indian summer restaurant
安德魯印度料理
老漁村餐廳
印度先生
旁遮普印度美食 Punjab Indian Restaurant
小魏川菜餐廳
伊莉莎海灘咖啡館
大王原汁牛肉麵
麥當勞-大雅中清南餐廳（設有得來速）
AL REVÉS 餐廳 /顛倒restaurant
旺旺來亞洲咖哩屋 HALAL FOOD
陳記蚵仔煎/麵/飯/合菜
瑛的快炒小吃店
香園客家小吃店
Anchor Restaurant 安可美式餐廳
都瓦哩印度厨房酒館 Doon valley Indian restaurant and bar
乳牛牧場景觀餐廳
Namaste Indian Restaurant 2 (景平店)
華星牛排館
啵妞咖啡/義式料理 BoNEW Restaurant & Café
金珮泰式料理
GREEN BUNNY restaurant
香料屋
品味黑白切
四哥的店
印度蘭異國餐廳Orchid Indian restaurant/barbecue/台北美食/台北異國料理/附近美食/台北印度料理
Hilans Indian Restaurant 依蘭斯印度餐廳
Hilans Indian Restaurant 依蘭斯印度餐廳
喵掌櫃咖哩丼飯
魔法咖哩 淡水店
金山鼎鮮美食館
東眼山食堂
藍天印度烤餅
濃濃西餐廳
淇里思印度餐館 CHILLIESINE Indian Restaurant 大坑店 Dakeng
門前隱味牛肉麵(搬到三重空軍一村)
馬友友印度廚房 - 大直 Mayur Indian Kitchen restaurant Dazhi (MiK-hi5)
胖老爹美式炸雞 金山店
此木窯 Chai Space & Store-萬里咖啡 萬里coffee 萬里窯烤麵包 萬里窯烤pizza 萬里寵物友善景觀餐廳 金山萬里美食
名月居平價小吃店
丸居酒屋
老塘湖藝術村鹽鄉館
Happiness．遇見幸福．
正德居
飯丸屋-桃園中壢店 ｜沖繩飯糰 ｜異國美食｜沖繩美食
廟口麵店
Saüté Restaurant 索緹餐酒
鑫園便當店
點22 港式點心專賣店-三峽民生店
白圍牆地中海景觀咖啡餐廳-桃園景觀

In [18]:
!wget https://github.com/notofonts/noto-cjk/blob/main/Sans/OTC/NotoSansCJK-Regular.ttc

--2024-05-17 05:28:53--  https://github.com/notofonts/noto-cjk/blob/main/Sans/OTC/NotoSansCJK-Regular.ttc
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘NotoSansCJK-Regular.ttc’

NotoSansCJK-Regular     [ <=>                ] 260.21K  1.46MB/s    in 0.2s    

2024-05-17 05:28:54 (1.46 MB/s) - ‘NotoSansCJK-Regular.ttc’ saved [266457]



In [ ]:
# 使用範例
chosen_restaurant = "籌學費餐廳	"
recommendations = get_restaurant_recommendations(chosen_restaurant, G)
print(f"對於 '{chosen_restaurant}' 的推薦餐廳:")
for restaurant in recommendations:
    print(restaurant)

對於 '籌學費餐廳' 的推薦餐廳:
名傳牛排館
山嵐拉麵 公館店
柴寮披薩 Luau pizza
泰式料理- PATTAYA
China Spice 聚味軒中餐廳
吉品勝口味
南郊農家菜館
爭鮮迴轉壽司-內湖店
Janta South Indian Restaurant
竹興早餐店


In [ ]:
df = df_tour
df_outliner = df[df['rating'] > 3.0]
Q3_user_ratings_total = np.percentile(df_outliner['user_ratings_total'], 10)
df_outliner_total = df_outliner[df_outliner['user_ratings_total'] > Q3_user_ratings_total]
df = df_outliner_total
dif = ['台北市中正區', '台北市大同區', '台北市中山區', '台北市松山區', '台北市大安區', '台北市萬華區', '台北市信義區', '台北市士林區', '台北市北投區', '台北市南港區', '台北市內湖區',
       '台北市文山區','新北市板橋區','新北市中和區','新北市新莊區','新北市土城區','新北市汐止區','新北市鶯歌區','新北市淡水區','新北市五股區','新北市林口區','新北市深坑區','新北市坪林區','新北市石門區','新北市萬里區',
       '新北市雙溪區','新北市烏來區','新北市三重區','新北市永和區','新北市新店區','新北市蘆洲區','新北市樹林區','新北市三峽區','新北市瑞芳區','新北市泰山區','新北市八里區','新北市石碇區','新北市三芝區','新北市金山區','新北市平溪區','新北市貢寮區'
       ,'桃園市中壢區','桃園市平鎮區','桃園市龍潭區','桃園市楊梅區','桃園市新屋區','桃園市觀音區','桃園市桃園區','桃園市龜山區','桃園市八德區','桃園市大溪區','桃園市復興區','桃園市大園區','桃園市蘆竹區',
       '基隆市中山區','基隆市中正區','基隆市信義區','基隆市仁愛區','基隆市暖暖區','基隆市七堵區','基隆市安樂區']

def match_address(row):
    for district in dif:
        if district in row['formatted_address']:
            return district
    return None

df['district'] = df.apply(match_address, axis=1)
matched_places = df[df['district'].notnull()]


<ipython-input-7-ddc9af882f52>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['district'] = df.apply(match_address, axis=1)


In [ ]:
import json
import ast
from collections import defaultdict

def match_time(row):
    # Handle missing data
    if pd.isna(row['opening_hours']):
        return None

    try:
        # Convert JSON-like string to dictionary if necessary
        if isinstance(row['opening_hours'], str):
            opening_hours = ast.literal_eval(row['opening_hours'])
        else:
            opening_hours = row['opening_hours']
    except (json.JSONDecodeError, ValueError):
        print(f"Failed to decode JSON for row: {row['opening_hours']}")
        return None

    # Initialize a dictionary to store time categories for each day
    day_categories = {day: [] for day in range(7)}

    # Handle the case where the 'periods' list contains a single dictionary with 'open': {'day': 0, 'time': '0000'}
    if 'open_now' in opening_hours and opening_hours['open_now'] and len(opening_hours['periods']) == 1:
        period = opening_hours['periods'][0]
        if 'open' in period and period['open']['time'] == '0000':
            for day in range(7):
                day_categories[day].append('全天')

    # Process each day's periods to determine the matching time category
    for period in opening_hours.get('periods', []):
        if 'open' in period and 'close' in period:
            # Ensure both open and close keys are present
            open_time = int(period['open']['time'])
            close_time = int(period['close']['time'])
            day = period['open']['day']

            # Check if the current period overlaps with any existing categories
            existing_categories = day_categories[day]
            if any(category in ['全天', '晚上'] for category in existing_categories):
                continue  # Skip if the day already has a higher priority category

            # Prioritize time categories
            if open_time == 0 and close_time == 2400:
                day_categories[day] = ['全天']
            elif open_time < 1200 and close_time >= 1800:
                day_categories[day] = ['早上', '下午']
            elif open_time >= 1800 or close_time >= 2400:
                day_categories[day].append('晚上')
            elif open_time < 1200:
                if '下午' not in day_categories[day]:
                    day_categories[day].append('早上')
            elif 1200 <= open_time < 1800:
                if '早上' not in day_categories[day]:
                    day_categories[day].append('下午')

    # Join the time categories for each day with a comma separator
    day_categories = {day: ', '.join(set(categories)) for day, categories in day_categories.items()}

    return day_categories

In [ ]:
geolocator = Nominatim(user_agent="geotest")

def extract_lat(location):
    lat_lng_dict = eval(location)
    return lat_lng_dict['lat']

def extract_lng(location):
    lat_lng_dict = eval(location)
    return lat_lng_dict['lng']

df['lat'] = df['location(lat,lng)'].apply(extract_lat)
df['lng'] = df['location(lat,lng)'].apply(extract_lng)

<ipython-input-9-9386c953229d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat'] = df['location(lat,lng)'].apply(extract_lat)
<ipython-input-9-9386c953229d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lng'] = df['location(lat,lng)'].apply(extract_lng)


In [ ]:
df[['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日']] = df.apply(match_time, axis=1, result_type='expand')
df = df[['name', 'rating', 'user_ratings_total', '星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日', 'district','lat','lng']]


<ipython-input-10-d366357fee53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日']] = df.apply(match_time, axis=1, result_type='expand')
<ipython-input-10-d366357fee53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日']] = df.apply(match_time, axis=1, result_type='expand')
<ipython-input-10-d366357fee53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using